In [1]:
# CL1 + CL2 + FCL
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

tf.set_random_seed(1234)  
mnist = input_data.read_data_sets("mnist/", one_hot=True)

# hyper parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

# input place holders
X = tf.placeholder(tf.float32, [None, 784])
# input image reshape
X_img = tf.reshape(X, [-1, 28, 28, 1]) # img 28x28x1(black/white)    -1 -> replace to the num of sample
Y = tf.placeholder(tf.float32, [None, 10])


Extracting mnist/train-images-idx3-ubyte.gz
Extracting mnist/train-labels-idx1-ubyte.gz
Extracting mnist/t10k-images-idx3-ubyte.gz
Extracting mnist/t10k-labels-idx1-ubyte.gz


In [2]:

# 합성곱 1층 
W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))    # w,h,c,filter num: 채널수를 일치 시킨다!
L1 = tf.nn.conv2d(X_img, W1, strides=[1, 1, 1, 1], padding='SAME')   # ?, 28, 28, 32 (입력 size 동일)
L1 = tf.nn.relu(L1)   # shape 변경 없음
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], 
                    strides=[1, 2, 2, 1], padding='SAME') # ?, 14, 14, 32
'''
Tensor("Conv2D:0", shape=(?, 28, 28, 32), dtype=float32)
Tensor("Relu:0", shape=(?, 28, 28, 32), dtype=float32)
Tensor("MaxPool:0", shape=(?, 14, 14, 32), dtype=float32)
'''

# 합성곱 2층
W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))  # 32 채널수 똑같이 맞춰준다
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1],
                    strides=[1, 2, 2, 1], padding='SAME')

L2_flat = tf.reshape(L2, [-1, 7 * 7 * 64])  # 4차원 -> 2차원으로 변경 (n, 3136)   7 * 7* 64 = 3136
'''
Tensor("Conv2D_1:0", shape=(?, 14, 14, 64), dtype=float32)
Tensor("Relu_1:0", shape=(?, 14, 14, 64), dtype=float32)
Tensor("MaxPool_1:0", shape=(?, 7, 7, 64), dtype=float32)
Tensor("Reshape_1:0", shape=(?, 3136), dtype=float32)
'''

# 전결합층(fully connected layer) 
W3 = tf.get_variable("W3", shape=[7 * 7 * 64, 10],
                     initializer=tf.contrib.layers.xavier_initializer())  # L2의 col(3136)과 맞춰준다. 10 은 label수
b = tf.Variable(tf.random_normal([10]))
logits = tf.matmul(L2_flat, W3) + b


In [3]:

# define cost/loss & optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)


In [ ]:

# initialize
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# train my model
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = 500

    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feed_dict = {X: batch_xs, Y: batch_ys}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning Finished!')
# Test model and check accuracy
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy:', sess.run(accuracy, feed_dict={
      X: mnist.test.images, Y: mnist.test.labels}))

Learning started. It takes sometime.
Epoch: 0001 cost = 0.413437262
Epoch: 0002 cost = 0.110832769
Epoch: 0003 cost = 0.082987733
Epoch: 0004 cost = 0.065561283
Epoch: 0005 cost = 0.056926463
Epoch: 0006 cost = 0.051567470
Epoch: 0007 cost = 0.043593327
Epoch: 0008 cost = 0.039839921
Epoch: 0009 cost = 0.036237492
Epoch: 0010 cost = 0.029704944
Epoch: 0011 cost = 0.029170399
Epoch: 0012 cost = 0.024703394
Epoch: 0013 cost = 0.022162806
Epoch: 0014 cost = 0.019278782
Epoch: 0015 cost = 0.017932454
Learning Finished!
